In [1]:
import pandas as pd
import numpy as np

In [68]:
#keyword embedding
import io
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # warnings for tf
import re
import shutil
import string

#pip install tensorflow
import tensorflow as tf

from tensorflow.keras import Sequential, layers
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers import TextVectorization
#gradient boosting
#random forest classification

In [3]:
#import right-wing news dataset
brainded_right = pd.read_csv('/home/zoetustain/code/zulu-tango/news_and_echo_bubbles/raw_data/braindedright.csv')
#convert date+time column into separate columns
brainded_right[['pdate','time']] = brainded_right['pdate'].str.split(' ', n=1, expand=True)
brainded_right['pdate'] = pd.to_datetime(brainded_right['pdate'])

# we only want relatively recent news
mask = brainded_right['pdate'] > '2020-01-01'
brainded_right = brainded_right[mask].reset_index()

# import left-wing news dataset
brainded_left = pd.read_csv('/home/zoetustain/code/zulu-tango/news_and_echo_bubbles/raw_data/braindedleft.csv')
brainded_left[['pdate','time']] = brainded_left['pdate'].str.split(' ', n=1, expand=True)
brainded_left['pdate'] = pd.to_datetime(brainded_left['pdate'])

mask = brainded_left['pdate'] > '2020-01-01'
brainded_left = brainded_left[mask].reset_index()

# drop empty rows
brainded_right = brainded_right.dropna().reset_index()
brainded_left = brainded_left.dropna().reset_index()

In [4]:
brainded_left.drop(columns={'level_0','index','Unnamed: 0'},inplace=True)

In [5]:
brainded_right.drop(columns={'level_0','index','Unnamed: 0'},inplace=True)

# Prepare data

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
#add classification column to data
brainded_left['classifier'] = 0

In [8]:
brainded_right['classifier'] = 1

In [9]:
data_left = brainded_left[['text','classifier']]

In [10]:
data_right = brainded_right[['text','classifier']]

In [11]:
data_combined = data_left.merge(data_right,how='outer')

In [88]:
data_combined

,text,classifier
0,MSNBC host Mehdi Hasan went after Rep. Dan Cre...,0
1,Rep. Dan Crenshaw's (R-Texas) stock purchase h...,0
2,Sen. Ed Markey is the Massachusetts Democrat w...,0
3,"On Valentine's Day, the major utility that sup...",0
4,"On Valentine’s Day, the major utility that sup...",0
...,...,...
2876,Conservatives were bracing for the defection o...,1
2877,I rise today to offer a few thoughts about the...,1
2878,(mj0007/Getty Images)\n\nI’m still not sure wh...,1
2879,Much ink has been spilled about the Supreme Co...,1


In [89]:
X = data_combined[['text']]
y = data_combined.classifier

# Text Vectorisation

In [90]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [97]:
# Instantiating the TfidfVectorizer
tf_idf_vectorizer = TfidfVectorizer(max_df = 0.98, min_df = 3)

In [96]:
# Instantiate the CountVectorizer with max_df = 2
count_vectorizer = CountVectorizer(max_df = 1000) 

# Train it
data_X = count_vectorizer.fit_transform(X.text)
data_X = pd.DataFrame(
    data_X.toarray(),
    columns = count_vectorizer.get_feature_names_out(),
    index = X.text
)

data_X

00  000  0000  00004  \
text                                                                       
MSNBC host Mehdi Hasan went after Rep. Dan Cren...   0    0     0      0   
Rep. Dan Crenshaw's (R-Texas) stock purchase hi...   0    3     0      0   
Sen. Ed Markey is the Massachusetts Democrat wh...   0    0     0      0   
On Valentine's Day, the major utility that supp...   0    3     0      0   
On Valentine’s Day, the major utility that supp...   0    3     0      0   
...                                                 ..  ...   ...    ...   
Conservatives were bracing for the defection of...   0    0     0      0   
I rise today to offer a few thoughts about the ...   0    0     0      0   
(mj0007/Getty Images)\n\nI’m still not sure whi...   0    0     0      0   
Much ink has been spilled about the Supreme Cou...   0    0     0      0   
We conservatives now find ourselves trying to p...   0    0     0      0   

                                                    0001177  0002  00032597  \
text                                                                          
MSNBC host Mehdi Hasan went after Rep. Dan Cren...        0     0         0   
Rep. Dan Crenshaw's (R-Texas) stock purchase hi...        0     0         0   
Sen. Ed Markey is the Massachusetts Democrat wh...        0     0         0   
On Valentine's Day, the major utility that supp...        0     0         0   
On Valentine’s Day, the major utility that supp...        0     0         0   
...                                                     ...   ...       ...   
Conservatives were bracing for the defection of...        0     0         0   
I rise today to offer a few thoughts about the ...        0     0         0   
(mj0007/Getty Images)\n\nI’m still not sure whi...        0     0         0   
Much ink has been spilled about the Supreme Cou...        0     0         0   
We conservatives now find ourselves trying to p...        0     0         0   

                                                    00075  00088618  0009  \
text                                                                        
MSNBC host Mehdi Hasan went after Rep. Dan Cren...      0         0     0   
Rep. Dan Crenshaw's (R-Texas) stock purchase hi...      0         0     0   
Sen. Ed Markey is the Massachusetts Democrat wh...      0         0     0   
On Valentine's Day, the major utility that supp...      0         0     0   
On Valentine’s Day, the major utility that supp...      0         0     0   
...                                                   ...       ...   ...   
Conservatives were bracing for the defection of...      0         0     0   
I rise today to offer a few thoughts about the ...      0         0     0   
(mj0007/Getty Images)\n\nI’m still not sure whi...      0         0     0   
Much ink has been spilled about the Supreme Cou...      0         0     0   
We conservatives now find ourselves trying to p...      0         0     0   

                                                    ...  ﬁlled  ﬁnd  ﬁngers  \
text                                                ...                       
MSNBC host Mehdi Hasan went after Rep. Dan Cren...  ...      0    0       0   
Rep. Dan Crenshaw's (R-Texas) stock purchase hi...  ...      0    0       0   
Sen. Ed Markey is the Massachusetts Democrat wh...  ...      0    0       0   
On Valentine's Day, the major utility that supp...  ...      0    0       0   
On Valentine’s Day, the major utility that supp...  ...      0    0       0   
...                                                 ...    ...  ...     ...   
Conservatives were bracing for the defection of...  ...      0    0       0   
I rise today to offer a few thoughts about the ...  ...      0    0       0   
(mj0007/Getty Images)\n\nI’m still not sure whi...  ...      0    0       0   
Much ink has been spilled about the Supreme Cou...  ...      0    0       0   
We conservatives now find ourselves trying to p...  ...      0    0       0   

     

In [98]:
# Training it on the texts
weighted_words = pd.DataFrame(tf_idf_vectorizer.fit_transform(X.text).toarray(),
                 columns = tf_idf_vectorizer.get_feature_names_out())

weighted_words

,00,000,01,02,0274,03,035,04,05,06,...,zoonotic,zte,zuckerberg,zuma,às,áñez,état,être,órgão,única
0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.071799,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.015113,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.015113,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2876,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2877,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2878,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2879,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [99]:
X_train, X_test, y_train, y_test = train_test_split(weighted_words,y,test_size=0.2)

# Gradient Boosting Classifier

In [38]:
from sklearn.ensemble import GradientBoostingClassifier

In [100]:
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=1,random_state=0)

In [101]:
clf.fit(X_train,y_train)

GradientBoostingClassifier(learning_rate=1.0, max_depth=1, random_state=0)

In [102]:
clf.score(X_test, y_test)

0.7642980935875217

# Random Forest Classification

In [42]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [107]:
X, y = make_classification(n_samples=1000, n_features=24000,
                            n_informative=2, n_redundant=0,
                            random_state=0, shuffle=False)

In [108]:
clf = RandomForestClassifier(max_depth=2, random_state=0)

In [109]:
clf.fit(X_train, y_train)

RandomForestClassifier(max_depth=2, random_state=0)

In [110]:
clf.score(X_test,y_test)

0.5823223570190641